<a href="https://colab.research.google.com/github/yemaedahrav/CNN_Accelerators/blob/main/CIFAR_Lenet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
import torchvision.transforms as T
from torchvision.utils import make_grid
from torchvision.utils import save_image
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.optim import lr_scheduler

import time
import os
import shutil
import copy
import sys

import PIL
from IPython.display import Image
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans, MiniBatchKMeans
from statistics import mean
from collections  import OrderedDict
import numpy as np
from skimage import io, transform
import random
import scipy
import cv2
from math import floor, ceil
from datetime import datetime

!pip install torchinfo
from torchinfo import summary
%matplotlib inline
DEVICE = 'cpu'

In [2]:
%config InlineBackend.figure_format = 'retina'

In [3]:
transform = transforms.Compose([transforms.Resize((32, 32)),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                transforms.Grayscale(num_output_channels=1), 
                               ])

train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)

test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)

classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
RANDOM_SEED = 42
LEARNING_RATE = 0.001
BATCH_SIZE = 100
N_EPOCHS = 15
IMG_SIZE = 32
N_CLASSES = 10

In [5]:
def get_accuracy(model, data_loader, device):

    correct_pred = 0 
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            X = X.to(device)
            y_true = y_true.to(device)

            _, y_prob = model(X)
            _, predicted_labels = torch.max(y_prob, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n

In [6]:
def train(train_loader, model, criterion, optimizer, device):

    model.train()
    running_loss = 0
    
    for X, y_true in train_loader:

        optimizer.zero_grad()
        
        X = X.to(device)
        y_true = y_true.to(device)
    
        # Forward pass
        y_hat, _ = model(X) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

        # Backward pass
        loss.backward()
        optimizer.step()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss

In [7]:
def training_loop(model, criterion, optimizer, train_loader, epochs, device, print_every=1):

    best_loss = 1e10
    train_losses = []

    for epoch in range(0, epochs):

        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
        train_losses.append(train_loss)

        if epoch % print_every == (print_every - 1):
            
            train_acc = get_accuracy(model, train_loader, device=device)
                
            print(f'{datetime.now().time().replace(microsecond=0)} --- '
                  f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Train accuracy: {100 * train_acc:.2f}')
    
    return model, optimizer, train_losses

In [8]:
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=84, kernel_size=5, stride=1),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=84, out_features=64),
            nn.Tanh(),
            nn.Linear(in_features=64, out_features=n_classes),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs

In [9]:
torch.manual_seed(RANDOM_SEED)
model = LeNet5(N_CLASSES)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [10]:
#model, optimizer, _ = training_loop(model, criterion, optimizer, trainloader, N_EPOCHS, DEVICE)

In [11]:
for epoch in range(15):
    running_loss = 0.0
    start_time = time.time()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0], data[1]
        optimizer.zero_grad()
        logits, output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        end_time = time.time()
        time_taken = end_time - start_time

        running_loss += loss.item()
        if i % 2000 == 1999:    
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            print('Time:',time_taken)
            running_loss = 0.0

print('Finished Training of LeNet')

[1,  2000] loss: 2.224
Time: 18.368669509887695
[1,  4000] loss: 2.156
Time: 32.56694984436035
[1,  6000] loss: 2.117
Time: 46.82336616516113
[1,  8000] loss: 2.084
Time: 61.35168170928955
[1, 10000] loss: 2.077
Time: 76.23940467834473
[1, 12000] loss: 2.067
Time: 90.36059737205505
[2,  2000] loss: 2.047
Time: 14.348697662353516
[2,  4000] loss: 2.041
Time: 28.631542205810547
[2,  6000] loss: 2.038
Time: 42.99597215652466
[2,  8000] loss: 2.019
Time: 57.37296104431152
[2, 10000] loss: 2.008
Time: 71.67338442802429
[2, 12000] loss: 2.016
Time: 85.99613833427429
[3,  2000] loss: 1.994
Time: 14.197701215744019
[3,  4000] loss: 1.998
Time: 28.392746925354004
[3,  6000] loss: 1.991
Time: 42.632025718688965
[3,  8000] loss: 1.991
Time: 56.84504199028015
[3, 10000] loss: 1.990
Time: 71.0653395652771
[3, 12000] loss: 2.002
Time: 85.32175707817078
[4,  2000] loss: 1.964
Time: 14.33860182762146
[4,  4000] loss: 1.974
Time: 30.301779985427856
[4,  6000] loss: 1.963
Time: 44.64423727989197
[4,  80

In [12]:
test_accuracy = get_accuracy(model, testloader, DEVICE)
print('Accuracy of the network on the 10000 test images:', test_accuracy)

Accuracy of the network on the 10000 test images: tensor(0.5212)


In [14]:
from google.colab import files

torch.save(model.state_dict(), 'LeNet_CIFAR.pth')
files.download('LeNet_CIFAR.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>